In [171]:
%run input/Format.ipynb
import ROOT as root
from array import array
from math import sin
from ROOT import gRandom
from ROOT import TFile
from ROOT import TPad, TLegend
root.gErrorIgnoreLevel = root.kFatal
%jsroot on

In [172]:
iOption0 = 0
is_sigmalised = 0
sim = 1
new = 1

In [173]:
isigma = 2
N_files = 1
N_centr = 8
N_hists = 4
N_layers = 6
N_ilyaers = 1
is_e = 1
centralities = ["1-2 layer pos","1-2 layer neg","2-3 layer pos","2-3 layer neg","2-4 layer pos","2-4 layer neg","3-4 layer pos","3-4 layer neg","20-40%","40-60%","60-80%","80-93%"]
centrality = [1,1,2,2,3,3,4,4,5,5,6,6]
layer_bins = [35,36,40,41,47,48,52,53,59,60,64,65]
layer_bins = [91,91,96,96,100,100,101,101,105,105,110,110]
if is_sigmalised: layer_bins = [45,45,46,46,48,50,51,53,55,55,56,56]
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
file_path="input/rg/"
file_names="my-27th_QA_19200_350runs" #my-25th_QA_19197_1000runs
if new:
    file_path="input/NewHadron/"
    file_names="missalign_hadron_new4_rg0-8"
hist_select_3D_names = ["dphi_hist","dthe_hist","sdphi_hist","sdthe_hist"]
if is_e == 1: hist_select_3D_names = ["dphi_hist_el_dynamic","dthe_hist_el_dynamic","sdphi_hist_el_dynamic","sdthe_hist_el_dynamic"]
iOptames=["phi","the"]

In [174]:
if sim :
    file_path="../sim/"
    file_names="my-kek"

In [175]:
hists_emc_read = []

for iFile in range(N_files):
    print(file_path+file_names+f"{iFile}.root")
    if not new: infile = root.TFile.Open(file_path+file_names+f"{iFile}.root", "read")
    else: infile = root.TFile.Open(file_path+file_names+f".root", "read")
    hists_emc_read0 = []
    for icentr in range(N_centr):
        hist_select_3D0 = []
        for ihist in range(len(hist_select_3D_names)):
            hist_select_3D0.append(infile.Get(hist_select_3D_names[ihist]+f"_{icentr}"))
            hist_select_3D0[-1].SetDirectory(root.nullptr)
        hists_emc_read0.append(hist_select_3D0)
    hists_emc_read.append(hists_emc_read0)

infile.Close()

../sim/my-kek0.root


In [176]:
pt = array( 'd' )
pt_boarders = array( 'd', [0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,1.4,2.0,2.8,4.9])
if is_sigmalised: pt_boarders = array( 'd', [0.2,0.5,0.8,1.2,1.6,2.0,2.4,3.0,3.8,4.9])
pt_err = array( 'd' )
N_pt = len(pt_boarders) - 1
for i in range(N_pt):
    pt.append( (pt_boarders[i]+pt_boarders[i+1])/2)
    pt_err.append((pt_boarders[i+1]-pt_boarders[i])/2)
print("pt = ", pt,"\npt_boarders = ",pt_boarders)
if not is_sigmalised:  pt[0]=0.225

pt =  array('d', [0.25, 0.35, 0.45, 0.55, 0.6499999999999999, 0.75, 0.8500000000000001, 0.95, 1.2, 1.7, 2.4, 3.85]) 
pt_boarders =  array('d', [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.4, 2.0, 2.8, 4.9])


In [177]:
boards_tot = [[-0.099,0.099],[-0.099,0.099]]
boards_fit = [[-0.011,0.012],[-0.011,0.012]]
boards_fit2 = [[-0.03,-0.01],[0.015,0.035],[-0.09,-0.03],[0.035,0.09],[-0.09,-0.03],[0.035,0.09],[-0.06,-0.025],[0.03,0.065]]
boards_fit3 = [[-0.02,0.02],[-0.02,0.02]]
if is_sigmalised==1:
    boards_tot = [[-5,5],[-5,5]]
    boards_fit = [[-1,1],[-1,1]]
if is_sigmalised == 1:
    boards_tot = [[-10,10],[-10,10]]
    boards_fit = [[-1,1],[-1,1]]
    boards_fit2 = [[-6,-3],[3,6],[-6,-2],[2,6],[-6,-2],[2,6],[-6,-2],[2,6]]
    boards_fit3 = [[3,7],[-7,-3]]

In [178]:
def emc_fitter(InHist, ilayer, seed=1, icentr=0, sigma=2, fits_mean=0, fits_sigma=0):
    N_gaus=2
    seed0 = seed
    seed=seed*N_pt*icentr*100

    hist_pt_bin = InHist.ProjectionZ("kek")

    central_bean_prev = (0.001*layer_bins[2*ilayer]-0.0005-0.1)
    if is_sigmalised : central_bean_prev = 0
    Mean_local =  array( 'd' )
    Sigma_local =  array( 'd' )
    Mean_local_err =  array( 'd' )
    Sigma_local_err =  array( 'd' )

    int_fg_local =  array( 'd' )
    int_bg_local =  array( 'd' )
 
    for p in range(N_pt):
        c1 = root.TCanvas(f"c1{seed+p}",f"c1{seed+p}",720,360)

        m2_name=str(icentr)+"/"+str(ilayer)+"dynamic_"+str(round(pt[p],1))

        emc_var_porj = InHist.ProjectionX(f"emc{seed+p}_{seed+p}_{icentr}_{ilayer}",layer_bins[2*ilayer],layer_bins[2*ilayer+1],hist_pt_bin.FindBin(pt_boarders[p]),hist_pt_bin.FindBin(pt_boarders[p+1])-1)
        emc_var_porj.SetLineColor(1)

        par_tot = array( 'd', (3*N_gaus+3)*[2.] )
        g_gaus= root.TF1( f'g_gaus_{seed+p}_{icentr}_{ilayer}', 'gaus',  boards_fit[int(icentr%2)][0]+central_bean_prev, boards_fit[int(icentr%2)][1]+central_bean_prev)
        g_gaus2= root.TF1( f'g_gaus_2_{seed+p}_{icentr}_{ilayer}', 'gaus',  boards_fit2[icentr][0]+central_bean_prev, boards_fit2[icentr][1]+central_bean_prev)#[0]/[2]/sqrt(2*3.14159)*exp(-0.5*pow((x-[1])/[2],2))
        g_pol = root.TF1( f'g_pol{seed+p}_{icentr}_{ilayer}', '[0]+[1]*x*(abs(x)/x-1)+[2]*x*(abs(x)/x+1)',  boards_tot[int(icentr%2)][0],boards_tot[int(icentr%2)][1])
        Format_Func(g_pol,3,2)

        total = root.TF1( f'total_pos{seed+p}_{icentr}_{ilayer}', 'gaus(0)+gaus(3)+[6]+[7]*x*(abs(x)/x-1)+[8]*x*(abs(x)/x+1)',boards_tot[int(icentr%2)][0], boards_tot[int(icentr%2)][1])
        Format_Func(total,2,6)

        g_gaus.SetParameter(1,0.)
        g_gaus.SetParameter(2,boards_fit[1][1])
        g_gaus2.SetParameter(1,0.5*(boards_fit2[icentr][0]+boards_fit2[icentr][1])+central_bean_prev)
        g_gaus2.SetParameter(2,0.5*(boards_fit2[icentr][1]-boards_fit2[icentr][0]))
        g_gaus2.SetParLimits(0,1,10000000)
        g_gaus2.SetParLimits(1,boards_fit2[icentr][0]+central_bean_prev, boards_fit2[icentr][1]+central_bean_prev)

        emc_var_porj.Draw()
        emc_var_porj.Fit( g_gaus, 'QR')

        if iOption0 == 4:
            Mean_local.append(g_gaus.GetParameter(1))
            Sigma_local.append(g_gaus.GetParameter(2))
            Mean_local_err.append(g_gaus.GetParError(1)+0.05*g_gaus.GetParameter(2))
            Sigma_local_err.append(g_gaus.GetParError(2)+0.05*g_gaus.GetParameter(2))
            c1.SaveAs( "output/d_delta_fits/" + m2_name + ".png" )
            continue

        emc_var_porj.Fit( g_gaus2, 'QR+' )
        emc_var_porj.Fit( g_pol, 'QR+' )

        par_gaus = g_gaus.GetParameters()
        par_gaus2 = g_gaus2.GetParameters()
        par_pol = g_pol.GetParameters()

        par_tot[0], par_tot[1], par_tot[2], par_tot[3], par_tot[4], par_tot[5] = par_gaus[0], par_gaus[1], par_gaus[2], par_gaus2[0], par_gaus2[1], par_gaus2[2]
        par_tot[6], par_tot[7], par_tot[8] = par_pol[0], par_pol[1], par_pol[1]

        total.SetParameters( par_tot )
        total.SetParLimits(1,boards_tot[int(icentr%2)][0]/10+central_bean_prev,boards_tot[int(icentr%2)][1]/10+central_bean_prev)
        total.SetParLimits(2,boards_fit[int(icentr%2)][1]/10,2)
        total.SetParLimits(3,par_gaus2[0]*0.49, par_gaus2[0]*2)
        total.SetParLimits(4,par_gaus2[1]-0.01, par_gaus2[1]+0.01)
        total.SetParLimits(5,par_gaus2[2]*0.49, par_gaus2[2]*2)
        #total.SetParLimits(3,1,1000000)
        if ilayer>-1:
            total.SetParLimits(1,boards_tot[int(icentr%2)][0],boards_tot[int(icentr%2)][1])
        #if iOption0%2==1: total.FixParameter(3,0.)
        
        emc_var_porj.Draw()
        emc_var_porj.Fit( total, 'QR' )

        par_tot = total.GetParameters()

        g_gaus= root.TF1( f'g_gaus_4_{seed+p}_{icentr}_{ilayer}', 'gaus',  par_tot[1]-2*par_tot[2], par_tot[1]+2*par_tot[2])
        g_gaus2= root.TF1( f'g_gaus_5_{seed+p}_{icentr}_{ilayer}', 'gaus',  par_tot[4]-2*par_tot[5], par_tot[4]+2*par_tot[5])

        Format_Func(g_gaus,3,4)
        Format_Func(g_gaus2,3,root.kGreen+2)

        par_gaus[0], par_gaus[1], par_gaus[2], par_gaus2[0], par_gaus2[1], par_gaus2[2] = par_tot[0], par_tot[1], par_tot[2], par_tot[3], par_tot[4], par_tot[5]
        par_pol[0], par_pol[1], par_pol[2] = par_tot[6], par_tot[7], par_tot[8]
        g_gaus2.SetParameters(par_gaus2)
        g_gaus.SetParameters(par_gaus)
        g_pol.SetParameters(par_pol)

        g_gaus.Draw('same')
        g_gaus2.Draw('same')
        g_pol.Draw('same')

        Mean_local.append(total.GetParameter(1))
        Sigma_local.append(total.GetParameter(2))
        if iOption0<4 and is_sigmalised==0:
            Mean_local_err.append(total.GetParError(1)+0.05*total.GetParameter(2))
            Sigma_local_err.append(total.GetParError(2)+0.05*total.GetParameter(2))
        else:
            Mean_local_err.append(total.GetParError(1)+ (0.15-0.005*p)*total.GetParameter(2))
            Sigma_local_err.append(total.GetParError(2)+(0.15-0.005*p)*total.GetParameter(2))


        int_fg_local.append(g_gaus.Integral(total.GetParameter(1)-sigma*total.GetParameter(2),total.GetParameter(1)+sigma*total.GetParameter(2)))
        int_bg_local.append(g_pol.Integral(total.GetParameter(1)-sigma*total.GetParameter(2),total.GetParameter(1)+sigma*total.GetParameter(2)))
            
        #m2_pos.Draw("same") m2_neg.Draw("same")

        c1.SaveAs( "output/dynamic_delta_fits/" + m2_name + ".png" )

    return [Mean_local, Mean_local_err, Sigma_local, Sigma_local_err, int_fg_local, int_bg_local]


In [179]:
def emc_centr_fitter(InHist, layers, seed=1, sigma=2, fits_mean=0, fits_sigma=0):
    out_hists = []
    for icentr in range(N_centr):
        out_hists1 = []
        for ilayer in layers:
            out_hists1.append(emc_fitter(InHist[seed][icentr][iOption0+is_sigmalised*2], ilayer, seed, icentr, sigma, fits_mean, fits_sigma))
        out_hists.append(out_hists1)
    return out_hists

In [180]:
import multiprocess as mp

Ntr = N_files

pool = mp.Pool(Ntr)
output_array = pool.starmap(emc_centr_fitter, [(hists_emc_read, [0,1,2,3,4,5], i, isigma) for i in range(Ntr)])
pool.close()

In [181]:
Means, Mean_errs, Sigmas, Sigma_errs, Int_FG, Int_BG = [], [], [], [], [], []
for ifile in range(Ntr):
    Means1, Mean_errs1, Sigmas1, Sigma_errs1, Int_FG1, Int_BG1 = [], [], [], [], [], []
    for icentr in range(N_centr):
        Means0, Mean_errs0, Sigmas0, Sigma_errs0, Int_FG0, Int_BG0 = [], [], [], [], [], []
        for ilayer in range(N_layers):
            Means0.append(output_array[ifile][icentr][ilayer][0])
            Mean_errs0.append(output_array[ifile][icentr][ilayer][1])
            Sigmas0.append(output_array[ifile][icentr][ilayer][2])
            Sigma_errs0.append(output_array[ifile][icentr][ilayer][3])
            Int_FG0.append(output_array[ifile][icentr][ilayer][4])
            Int_BG0.append(output_array[ifile][icentr][ilayer][5])
        Means1.append(Means0)
        Mean_errs1.append(Mean_errs0)
        Sigmas1.append(Sigmas0)
        Sigma_errs1.append(Sigma_errs0)
        Int_FG1.append(Int_FG0)
        Int_BG1.append(Int_BG0)
    Means.append(Means1)
    Mean_errs.append(Mean_errs1)
    Sigmas.append(Sigmas1)
    Sigma_errs.append(Sigma_errs1)
    Int_FG.append(Int_FG1)
    Int_BG.append(Int_BG1)

In [182]:
types = ["emc_r_dphi_","emc_min_r_dphi_","emc_cnt_dphi_","emc_r_dz_","emc_min_r_dz_","emc_cnt_dz_"]
fit_functions = []
for ifile in range(N_files):
    fit_functions0 = []
    for icentr in range(N_centr):
        fit_functions1 = []
        for ilayer in range(N_layers):
            fit_functions1.append(root.TF1(f"fit_functions_sigma_{ifile}_{ilayer}_{icentr}","[0]+[1]*exp([2]*x)",0.5,5.0))
            fit_functions1[-1].SetParameter(0,0.0015)
            fit_functions1[-1].SetParameter(1,0.02)
            fit_functions1[-1].SetParameter(2,-2)
            fit_functions1[-1].SetParLimits(2,-8.126,-0.023)
            fit_functions1[-1].SetParLimits(0,0.001,0.002)
            if icentr<2 : fit_functions1[-1].SetParLimits(2,-5.5+0.5,-4.4-0.5)
            elif icentr<4 : fit_functions1[-1].SetParLimits(2,-3.0+0.45,-2.0-0.45)
            else : 
                fit_functions1[-1].SetParLimits(2,-3.0+0.45,-2.0-0.45)
                fit_functions1[-1].SetParLimits(0,0.001,0.002)
            if iOption0%2==1: 
                fit_functions1[-1].SetParLimits(2,-5.126,-1.23)
                fit_functions1[-1].SetParameter(1,0.02)
                fit_functions1[-1].SetParLimits(1,0.001,0.6)
                fit_functions1[-1].SetParameter(0,0.005)
                fit_functions1[-1].SetParLimits(0,0.001,0.006)
            if is_sigmalised: 
                fit_functions1[-1].SetParLimits(0,0.,20)
                fit_functions1[-1].SetParLimits(1,0.,20)
                fit_functions1[-1].SetParLimits(2,-30,+10.23)
            fit_functions1[-1].SetLineColor(colors[ilayer])
        fit_functions0.append(fit_functions1)
    fit_functions.append(fit_functions0)

In [183]:

print(Sigmas[0][0][0],Sigmas[0][3][0])
for ifile in range(N_files):
    h2=[]
    c3 = root.TCanvas(f"c3{ifile}",f"c3{ifile}",1400,1800)
    c3.Divide(2,4)
    g_sigma = [[[[]]*N_layers]*N_centr]*N_files
    min, max = 0., 0.01
    if is_sigmalised==1: max, min = 1.99, 0.01
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for icentr in range(N_centr):
        c3.cd(icentr+1)
        h2.append(Format_Pad_old( 0.05, 5, min, max, "p_{T} (GeV/#it{c})", "Sigmas", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1))
        h2[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9,centralities[icentr]))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for ilayer in range(0,N_layers):
            g_sigma[ifile][icentr][ilayer].append(root.TGraphErrors(N_pt, pt, Sigmas[ifile][icentr][ilayer],pt_err,Sigma_errs[ifile][icentr][ilayer]))

            Format_Graph(g_sigma[ifile][icentr][ilayer][-1],20+int(ilayer/4),1, colors[ilayer], 3, colors[ilayer],1,1)
            g_sigma[ifile][icentr][ilayer][-1].Fit(fit_functions[ifile][icentr][ilayer],"Q","")
            g_sigma[ifile][icentr][ilayer][-1].Draw('P')
            central_bean_prev = round(0.002*layer_bins[2*ilayer]-0.001-0.1,3)
            legends[-1].AddEntry(g_sigma[ifile][icentr][ilayer][-1],f"phi_prev {central_bean_prev}","p")

        legends[-1].Draw()
    c3.Draw()
    c3.SaveAs(f"output/Sigmas/sigma_{ifile}{iOption0}{is_sigmalised}.png")

array('d', [0.00896966660350136, 0.001244221744510883, 0.0012000015501800261, 0.0018422641265037015, 0.0018923180246803609, 0.0014065760133087214, 0.0016299525909564978, 0.0013878594621339335, 0.0018886404820973256, 0.001200000176394747, 0.012, 0.012]) array('d', [0.022380567584305525, 0.003436286400908087, 0.001200032292044427, 0.0024578156783198342, 0.0023196086073265787, 0.0018517913919874525, 0.002069993514546762, 0.0019500316548188722, 0.001953589356987199, 0.002312623992507011, 0.0017998820236504608, 0.011291940462738391])


In [184]:
print(f"sigma_pt_params[{N_files}][{N_centr}][{N_layers}][3]="+"{")
for fit_functions_ifile in fit_functions:
    print("{")
    for fit_functions_icentr in fit_functions_ifile:
        print("{")
        for fits in fit_functions_icentr:
            print("{"+f"{round(fits.GetParameter(0),6)}, {round(fits.GetParameter(1),6)}, {round(fits.GetParameter(2),6)}"+"},")
        print("},") 
    print("},")      
print("};")        

sigma_pt_params[1][8][6][3]={
{
{
{0.001399, 0.015173, -5.0},
{0.00125, 0.035337, -5.0},
{0.001354, 0.010731, -4.999999},
{0.001782, 752431.583814, -5.0},
{0.002, 7.632063, -5.0},
{0.001122, 0.019597, -5.0},
},
{
{0.002, 3539657.662968, -5.0},
{0.001535, 0.04623, -5.0},
{0.001785, 0.001193, -5.0},
{0.001374, 0.004998, -4.900136},
{0.002, 5438.685296, -5.0},
{0.001995, 2293734.058839, -5.0},
},
{
{0.001, 141.968992, -2.55},
{0.001432, 0.003272, -2.45},
{0.001153, 0.010094, -2.55},
{0.00141, 0.002007, -2.45},
{0.001796, 0.005694, -2.55},
{0.001371, 0.00035, -2.45},
},
{
{0.00135, 0.017168, -2.55},
{0.001054, 0.042831, -2.55},
{0.001382, 0.004888, -2.55},
{0.001432, 0.003698, -2.45},
{0.001129, 0.215044, -2.55},
{0.001034, 0.008711, -2.55},
},
{
{0.001, 135.050674, -2.55},
{0.001562, 0.028149, -2.55},
{0.00155, 0.004106, -2.450001},
{0.00136, 0.005864, -2.55},
{0.001946, 0.002756, -2.45},
{0.00114, 0.004787, -2.55},
},
{
{0.00115, 0.018304, -2.55},
{0.001832, 0.004333, -2.55},
{0.00136, 0

In [185]:
fit_mean_functions = []
for ifile in range(N_files):
    fit_functions0 = []
    for icentr in range(N_centr):
        fit_functions1 = []
        for ilayer in range(N_layers):
            if not is_sigmalised: 
                fit_functions1.append(root.TF1(f"fit_functions_mean_{ifile}_{ilayer}_"+centralities[icentr],"[0]",0.5,5.0))
                fit_functions1[-1].SetParameter(0,0.029)
            else: 
                fit_functions1.append(root.TF1(f"fit_functions_mean_{ifile}_{ilayer}_"+centralities[icentr],"[0]+[1]*exp([2]*x)",0.0,5.0))
                if iOption0==0:
                    fit_functions1[-1].SetParLimits(2,-5.126,-0.23)
                    if icentr%2==0: 
                        fit_functions1[-1].SetParameter(1,-1.05)
                        fit_functions1[-1].SetParLimits(1,-2.0,-1)
                        fit_functions1[-1].SetParameter(0,0.8)
                        fit_functions1[-1].SetParLimits(0,0.1,1.3)
                    else: 
                        fit_functions1[-1].SetParameter(1,1.05)
                        fit_functions1[-1].SetParLimits(1,1,2.0)
                        fit_functions1[-1].SetParameter(0,-0.8)
                        fit_functions1[-1].SetParLimits(0,-1.3,-0.1)
                    if ilayer==3 and icentr ==7:  
                        fit_functions1[-1].SetParameter(0,-0.9)
                        fit_functions1[-1].SetParameter(1,1.6)
                        fit_functions1[-1].SetParameter(2,-0.9)
            fit_functions1[-1].SetLineColor(colors[ilayer])
        fit_functions0.append(fit_functions1)
    fit_mean_functions.append(fit_functions0)

In [186]:
for ifile in range(N_files):
    h2=[]
    c3 = root.TCanvas(f"c3{ifile}",f"c3{ifile}",1400,1800)
    c3.Divide(2,4)
    g_mean = [[[[]]*N_layers]*N_centr]*N_files
    min, max = -0.1, 0.1
    if is_sigmalised==1: max, min = 1.5, -1.5
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for icentr in range(N_centr):
        c3.cd(icentr+1)
        h2.append(Format_Pad_old( 0.05, 5, min, max, "p_{T} (GeV/#it{c})", "Means", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1))
        h2[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9,centralities[icentr]))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for ilayer in range(0,N_layers):

            g_mean[ifile][icentr][ilayer].append(root.TGraphErrors(N_pt, pt, Means[ifile][icentr][ilayer],pt_err,Mean_errs[ifile][icentr][ilayer]))
            
            Format_Graph(g_mean[ifile][icentr][ilayer][-1],20+int(ilayer/4),1, colors[ilayer], 3, colors[ilayer],1,1)
            if not is_sigmalised: g_mean[ifile][icentr][ilayer][-1].Fit(fit_mean_functions[ifile][icentr][ilayer],"Q","",0.7,1)
            else: g_mean[ifile][icentr][ilayer][-1].Fit(fit_mean_functions[ifile][icentr][ilayer],"Q","",0.3,5)
            g_mean[ifile][icentr][ilayer][-1].Draw('P')
            central_bean_prev = round(0.002*layer_bins[2*ilayer]-0.001-0.1,3)
            legends[-1].AddEntry(g_mean[ifile][icentr][ilayer][-1],f"phi_prev {central_bean_prev}","p")

        legends[-1].Draw()
    c3.Draw()
    c3.SaveAs(f"output/Sigmas/mean_{ifile}{iOption0}{is_sigmalised}.png")

In [187]:
if not is_sigmalised:
    print(f"mean_pt_params[{N_files}][{N_centr}][{N_layers}]="+"{")
    for fit_functions_file in fit_mean_functions:
        print("{")
        for fit_functions_syst in fit_functions_file:
            print("{"+f"{round(fit_functions_syst[0].GetParameter(0),6)}, {round(fit_functions_syst[1].GetParameter(0),6)},{round(fit_functions_syst[2].GetParameter(0),6)}, {round(fit_functions_syst[3].GetParameter(0),6)}, {round(fit_functions_syst[4].GetParameter(0),6)}, {round(fit_functions_syst[5].GetParameter(0),6)}"+"},")
        print("},")
    print("};")
else:
    print("const float "+iOptames[iOption0]+f"_sMean_pt_params[{N_files}][{N_centr}][3]="+"{")
    for fit_functions_ifile in fit_mean_functions:
        print("{")
        iicentr = 0
        for fits in fit_functions_ifile:
            print("{"+f"{round(fits[3-int(iicentr%2)].GetParameter(0),6)}, {round(fits[3-int(iicentr%2)].GetParameter(1),6)}, {round(fits[3-int(iicentr%2)].GetParameter(2),6)}"+"},")
            iicentr+=1
        print("},")      
    print("};")        
    print(f"sigma_pt_params[{N_files}][{N_centr}][{N_layers}][3]="+"{")
    for fit_functions_ifile in fit_mean_functions:
        print("{")
        for fit_functions_icentr in fit_functions_ifile:
            print("{")
            for fits in fit_functions_icentr:
                print("{"+f"{round(fits.GetParameter(0),6)}, {round(fits.GetParameter(1),6)}, {round(fits.GetParameter(2),6)}"+"},")
            print("},") 
        print("},")      
    print("};")        

mean_pt_params[1][8][6]={
{
{-0.009414, -0.004408,-0.000489, 0.000382, 0.004227, 0.00971},
{-0.009871, -0.004525,-0.000876, 0.000371, 0.003204, 0.008802},
{-0.01038, -0.004815,-0.000684, 0.000313, 0.004213, 0.009499},
{-0.009598, -0.00422,-0.000358, 0.001283, 0.005048, 0.010304},
{-0.011654, -0.00544,-0.00083, 0.000356, 0.005076, 0.011009},
{-0.011092, -0.00455,-0.000261, 0.000811, 0.00555, 0.010992},
{-0.010186, -0.004781,-0.000349, 0.014505, 0.00499, 0.056708},
{-0.010169, -0.004965,-0.009128, -0.074929, 0.004844, 0.010555},
},
};


In [188]:
dphi = array( 'd' )
dphi_err = array( 'd' )
dynamic_dphi = []
dynamic_dphi_err = []
for ilayer in range(N_layers):
    if not is_sigmalised: dphi.append( round(0.001*layer_bins[2*ilayer]-0.0005-0.1,5))
    else: dphi.append(round(0.2*layer_bins[2*ilayer]-0.1-10,5))
    dphi_err.append(0.001/4)

for ifile in range(N_files):
    dynamic_dphi_ifile = []
    dynamic_dphi_err_ifile = []
    for icentr in range(N_centr):
        dynamic_dphi_icentr = array( 'd' )
        dynamic_dphi_err_icentr = array( 'd' )
        for ilayer in range(N_layers):
            dynamic_dphi_icentr.append(fit_mean_functions[ifile][icentr][ilayer].GetParameter(0))
            dynamic_dphi_err_icentr.append(fit_mean_functions[ifile][icentr][ilayer].GetParError(0))
        dynamic_dphi_ifile.append(dynamic_dphi_icentr)
        dynamic_dphi_err_ifile.append(dynamic_dphi_err_icentr)
    dynamic_dphi.append(dynamic_dphi_ifile)
    dynamic_dphi_err.append(dynamic_dphi_err_ifile)

print("dphi = ", dphi)
print(dynamic_dphi)
if iOption0 != 0:
    dynamic_dphi[0][5][2]/=100
    dynamic_dphi[0][5][3]=0.002
    dynamic_dphi[0][4][3]=-0.0005
    dynamic_dphi[0][4][4]/=5

dphi =  array('d', [-0.0095, -0.0045, -0.0005, 0.0005, 0.0045, 0.0095])
[[array('d', [-0.009414306435111503, -0.004408170630863486, -0.0004891333123714717, 0.00038150597177047385, 0.004226965196783006, 0.009710334953555076]), array('d', [-0.009870779362938505, -0.004524570544561773, -0.0008763419847570286, 0.0003710167654527863, 0.003204305977639068, 0.008801716377406718]), array('d', [-0.010379761481724117, -0.0048145168107969726, -0.0006837973176017295, 0.0003130129799069799, 0.00421264321420961, 0.00949873754222226]), array('d', [-0.009598098107879507, -0.004219579155049089, -0.0003579236679388531, 0.0012834214865468097, 0.005048108540283234, 0.010304382733319364]), array('d', [-0.0116544590201292, -0.005439620054689159, -0.0008302118585127478, 0.0003556160105546609, 0.005075525105579699, 0.01100925593690557]), array('d', [-0.01109205970497543, -0.004549609155110549, -0.0002612954004478918, 0.0008112000765878868, 0.005550092485586869, 0.010992450083778332]), array('d', [-0.010186137

In [189]:
for ifile in range(N_files):
    h3=[]
    c4 = root.TCanvas(f"c4",f"c4",600,450)
    g_mean_phi = [[[[]]*N_layers]*N_centr]*N_files
    min, max, x0, x1 = -0.05, 0.05, -0.02, 0.05
    if is_sigmalised==1: max, min, x0, x1 = 2, -2, -2, 5
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for ilayer in range(1):
        h3.append(Format_Pad_old( x0, x1, min, max, "p_{T} (GeV/#it{c})", "Means", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1))
        h3[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for icentr in range(0,N_centr):

            g_mean_phi[ifile][icentr][ilayer].append(root.TGraphErrors(N_layers, dphi, dynamic_dphi[ifile][icentr],dphi_err,dynamic_dphi_err[ifile][icentr]))
            
            Format_Graph(g_mean_phi[ifile][icentr][ilayer][-1],20+int(icentr/4),1, colors[icentr], 3, colors[icentr],1,1)
            g_mean_phi[ifile][icentr][ilayer][-1].Fit("pol1","","")
            g_mean_phi[ifile][icentr][ilayer][-1].GetFunction("pol1").SetLineColor(colors[icentr])
            g_mean_phi[ifile][icentr][ilayer][-1].Draw('sameP')
            legends[-1].AddEntry(g_mean_phi[ifile][icentr][ilayer][-1],centralities[icentr],"p")

        legends[-1].Draw()
    c4.Draw()
    c4.SaveAs(f"output/Sigmas/mean_{ifile}{iOption0}{is_sigmalised}.png")

****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =         3544
NDf                       =            4
Edm                       =  7.13446e-06
NCalls                    =          302
p0                        =     -6.96286   +/-   2617.82     
p1                        = -2.56493e+07   +/-   1.17707e+06 
****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =         3544
NDf                       =            4
Edm                       =   5.5859e-08
NCalls                    =           24
p0                        = -0.000482458   +/-   2618.12     
p1                        = -2.56512e+07   +/-   8.81858e+06 
****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =         3544
NDf                       =            4
Edm                       =  1.20247e-07
NCalls                    =           30
p0                        = -0.000482458   +/- 

In [190]:
sigma_dynamic_dphi = []
sigma_dynamic_dphi_err = []
pt_point = 0.5

for ifile in range(N_files):
    dynamic_dphi_ifile = []
    dynamic_dphi_err_ifile = []
    for icentr in range(N_centr):
        dynamic_dphi_icentr = array( 'd' )
        dynamic_dphi_err_icentr = array( 'd' )
        for ilayer in range(N_layers):
            dynamic_dphi_icentr.append(fit_functions[ifile][icentr][ilayer].Eval(pt_point)/(fit_functions[ifile][icentr][2].Eval(pt_point)+fit_functions[ifile][icentr][3].Eval(pt_point))*2)
            dynamic_dphi_err_icentr.append((fit_functions[ifile][icentr][ilayer].GetParError(0)*2)/(fit_functions[ifile][icentr][2].Eval(pt_point)+fit_functions[ifile][icentr][3].Eval(pt_point))*2)
        dynamic_dphi_ifile.append(dynamic_dphi_icentr)
        dynamic_dphi_err_ifile.append(dynamic_dphi_err_icentr)
    sigma_dynamic_dphi.append(dynamic_dphi_ifile)
    sigma_dynamic_dphi_err.append(dynamic_dphi_err_ifile)

print(sigma_dynamic_dphi)
sigma_dynamic_dphi[0][5][0]=0.88

[[array('d', [8.563601208348084e-08, 1.3440510306566337e-07, 7.235901166157074e-08, 1.9999999276409883, 2.0351158369417998e-05, 8.843208636875459e-08]), array('d', [157572383.10873696, 2.89058026943953, 1.02110589532355, 0.9788941046764501, 242111.11909099726, 102108502.29132189]), array('d', [13281.504765502632, 0.800997185637144, 1.33039653157175, 0.6696034684282498, 1.1337559199934428, 0.4932817307132521]), array('d', [2.33456155224423, 4.945929862904346, 1.0435286047656487, 0.9564713952343511, 23.250708535745495, 1.3172151543197155]), array('d', [13115.012655107035, 3.276227414710017, 0.9578754110169659, 1.0421245889830342, 0.9576116503097899, 0.8609218650878167]), array('d', [1.9708395784921466, 0.9571480878248116, 1.0153383436530727, 0.9846616563469273, 1.340972980301812, 12115.965437791616]), array('d', [842.4489033440257, 979.8944554580861, 0.009663514858805898, 1.990336485141194, 0.09482878906123313, 4.892693426814995]), array('d', [0.3726955699606078, 0.41151733530863044, 1.4

In [191]:
for ifile in range(N_files):
    h3=[]
    c4 = root.TCanvas(f"c4",f"c4",600,450)
    g_sigma_phi = [[[[]]*N_layers]*N_centr]*N_files
    min, max, x0, x1 = 0, 2, -0.02, 0.05
    if is_sigmalised==1: max, min, x0, x1 = 4, -2, -2, 5
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for ilayer in range(1):
        h3.append(Format_Pad_old( x0, x1, min, max, "p_{T} (GeV/#it{c})", "Means", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1))
        h3[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for icentr in range(0,N_centr):

            g_sigma_phi[ifile][icentr][ilayer].append(root.TGraphErrors(N_layers, dphi, sigma_dynamic_dphi[ifile][icentr],dphi_err,sigma_dynamic_dphi_err[ifile][icentr]))
            
            Format_Graph(g_sigma_phi[ifile][icentr][ilayer][-1],20+int(icentr/4),1, colors[icentr], 3, colors[icentr],1,1)
            g_sigma_phi[ifile][icentr][ilayer][-1].Fit("pol2","","",)
            g_sigma_phi[ifile][icentr][ilayer][-1].GetFunction("pol2").SetLineColor(colors[icentr])
            g_sigma_phi[ifile][icentr][ilayer][-1].Draw('sameP')
            legends[-1].AddEntry(g_sigma_phi[ifile][icentr][ilayer][-1],centralities[icentr],"p")

        legends[-1].Draw()
    c4.Draw()
    c4.SaveAs(f"output/Sigmas/mean_{ifile}{iOption0}{is_sigmalised}.png")

****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      3545.55
NDf                       =            3
Edm                       =  2.39231e-07
NCalls                    =           82
p0                        = -6.88743e+17   +/-   2.224e+17   
p1                        = -2.18299e+21   +/-   5.40733e+16 
p2                        = -6.21432e+21   +/-   1.56398e+17 
****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      3545.55
NDf                       =            3
Edm                       =  5.83088e-29
NCalls                    =           26
p0                        = -6.88743e+17   +/-   2           
p1                        = -2.18299e+21   +/-   2           
p2                        = -6.21432e+21   +/-   2           
****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      3545.55
NDf                       =            3
Edm  

In [192]:
print("const float "+iOptames[iOption0]+f"_sigma_pt_params[{N_files}][{N_centr}][3]="+"{")
for fit_functions_ifile in fit_functions:
    print("{")
    for fits in fit_functions_ifile:
        print("{"+f"{round((fits[2].GetParameter(0)+fits[3].GetParameter(0))*0.5,6)}, {round((fits[2].GetParameter(1)+fits[3].GetParameter(1))*0.5,6)}, {round((fits[2].GetParameter(2)+fits[3].GetParameter(2))*0.5,6)}"+"},")
    print("},")      
print("};")
print("const float "+iOptames[iOption0]+f"_sigma_"+iOptames[iOption0]+f"_params[{N_files}][{N_centr}][3]="+"{")
for ifile in range(N_files):
    print("{")
    for icentr in range(N_centr):
        print("{"+f'{round(g_sigma_phi[ifile][icentr][0][icentr].GetFunction("pol2").GetParameter(0),6)},{round(g_sigma_phi[ifile][icentr][0][icentr].GetFunction("pol2").GetParameter(1),6)},{round(g_sigma_phi[ifile][icentr][0][icentr].GetFunction("pol2").GetParameter(2),6)}'+"},")
    print("},")
print("};")
print("const float "+iOptames[iOption0]+f"_mean_"+iOptames[iOption0]+f"_params[{N_files}][{N_centr}][2]="+"{")
for ifile in range(N_files):
    print("{")
    for icentr in range(N_centr):
        print("{"+f'{round(g_mean_phi[ifile][icentr][0][icentr].GetFunction("pol1").GetParameter(0),6)},{round(g_mean_phi[ifile][icentr][0][icentr].GetFunction("pol1").GetParameter(1),6)}'+"},")
    print("},")
print("};")



const float phi_sigma_pt_params[1][8][3]={
{
{0.001568, 376215.797272, -5.0},
{0.001579, 0.003095, -4.950068},
{0.001282, 0.006051, -2.5},
{0.001407, 0.004293, -2.5},
{0.001455, 0.004985, -2.5},
{0.00125, 0.00674, -2.500012},
{0.001923, 0.161891, -2.5},
{0.001644, 0.029538, -2.55},
},
};
const float phi_sigma_phi_params[1][8][3]={
{
{-6.887427958882747e+17,-2.182991864027171e+21,-6.214315963869224e+21},
{-6.887427958882747e+17,-2.182991864027171e+21,-6.214315963869224e+21},
{-6.887427958882747e+17,-2.182991864027171e+21,-6.214315963869224e+21},
{-6.887427958882747e+17,-2.182991864027171e+21,-6.214315963869224e+21},
{-6.887427958882747e+17,-2.182991864027171e+21,-6.214315963869224e+21},
{-6.887427958882747e+17,-2.182991864027171e+21,-6.214315963869224e+21},
{-6.887427958882747e+17,-2.182991864027171e+21,-6.214315963869224e+21},
{-6.887427958882747e+17,-2.182991864027171e+21,-6.214315963869224e+21},
},
};
const float phi_mean_phi_params[1][8][2]={
{
{-6.962858,-25649254.811769},
{-0.0004